<a href="https://colab.research.google.com/github/Minteb/Hyperspectral-Image-Classification-using-Random-Forest-Algorithm/blob/main/Landsat8_NDVI_2013_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Install required packages
!pip install earthengine-api geemap geopandas pycrs

# Import libraries
import ee
import geemap
import os
import zipfile
from google.colab import files
import geopandas as gpd
from datetime import datetime
import time
import io
from google.colab import drive

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='spiaproject')

# 1. SHAPEFILE UPLOAD AND PROCESSING
def upload_shapefile():
    print("Please upload your shapefile (as .zip containing .shp/.shx/.dbf/.prj)")
    uploaded = files.upload()

    if not uploaded:
        raise ValueError("No files were uploaded")

    # Process ZIP file
    zip_files = [f for f in uploaded.keys() if f.lower().endswith('.zip')]
    if not zip_files:
        raise ValueError("No ZIP file found in upload")

    try:
        with zipfile.ZipFile(zip_files[0], 'r') as zip_ref:
            shapefile_dir = '/content/shapefile/'
            os.makedirs(shapefile_dir, exist_ok=True)
            zip_ref.extractall(shapefile_dir)

            # Find the .shp file
            shp_files = [f for f in os.listdir(shapefile_dir) if f.lower().endswith('.shp')]
            if not shp_files:
                raise FileNotFoundError("No .shp file found in the uploaded files")

            return os.path.join(shapefile_dir, shp_files[0])
    except Exception as e:
        raise ValueError(f"Error processing shapefile: {str(e)}")

# Upload and process shapefile
shp_path = upload_shapefile()
print(f"Shapefile loaded: {shp_path}")

# Convert to EE geometry
roi = geemap.shp_to_ee(shp_path)
if roi is None:
    # Fallback method if standard conversion fails
    gdf = gpd.read_file(shp_path)
    roi = geemap.geopandas_to_ee(gdf)

print("ROI converted to Earth Engine geometry")

# 2. LANDSAT PROCESSING FUNCTIONS
def mask_clouds(image):
    """Mask clouds and cloud shadows in Landsat 8/9 images"""
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
           .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

def add_ndvi(image):
    """Calculate NDVI and add it as a band"""
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return image.addBands(ndvi)

def create_monthly_composites(year, month):
    """Create monthly median composite and calculate mean NDVI"""
    start_date = datetime(year, month, 1).strftime('%Y-%m-%d')
    if month == 12:
        end_date = datetime(year + 1, 1, 1).strftime('%Y-%m-%d')
    else:
        end_date = datetime(year, month + 1, 1).strftime('%Y-%m-%d')

    # Filter collection by date and region
    monthly_collection = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                         .filterBounds(roi)
                         .filterDate(start_date, end_date)
                         .map(mask_clouds)
                         .map(add_ndvi))

    # Create median composite
    composite = monthly_collection.median()

    # Calculate mean NDVI for the ROI
    mean_val = composite.select('NDVI').reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=30,
        maxPixels=1e9
    ).get('NDVI')

    return composite.select('NDVI'), mean_val

# 3. OPTIMIZED PROCESSING AND EXPORT
# Create output folder
output_folder = '/content/ndvi_output'
os.makedirs(output_folder, exist_ok=True)

# Create a list to store all export tasks
tasks = []

# Process each year and month
for year in range(2025, 2026):
    print(f"\nProcessing year: {year}")
    for month in range(1, 13):
        print(f"Month: {month:02d}", end=' ')

        # Skip months before Landsat 8 launch (April 2013)
        if year == 2013 and month < 4:
            print("⏭️ Skipped (pre-Landsat 8)")
            continue

        # Skip future months
        current_year = datetime.now().year
        current_month = datetime.now().month
        if year > current_year or (year == current_year and month > current_month):
            print("⏭️ Skipped (future date)")
            continue

        try:
            # Create monthly composite
            composite, mean_val = create_monthly_composites(year, month)
            mean_val_info = mean_val.getInfo()

            if mean_val_info is None:
                print("⚠️ No valid data")
                continue

            # Prepare filename
            filename = f"NDVI_{year}_{month:02d}.tif"

            # Start export task (but don't wait for completion)
            task = ee.batch.Export.image.toDrive(
                image=composite,
                description=f'NDVI_{year}_{month:02d}',
                folder='GEE_Exports',
                fileNamePrefix=filename.replace('.tif', ''),
                scale=30,
                region=roi.geometry(),
                maxPixels=1e9,
                fileFormat='GeoTIFF'
            )
            task.start()
            tasks.append(task)

            print(f"✅ Export started (Mean NDVI: {mean_val_info:.3f})")

        except Exception as e:
            print(f"❌ Error: {str(e)}")
            continue

# Monitor all tasks
print("\nMonitoring all export tasks...")
while len(tasks) > 0:
    for task in tasks[:]:  # Make a copy for iteration
        status = task.status()
        state = status['state']

        if state == 'COMPLETED':
            print(f"Task {task.id} completed successfully!")
            tasks.remove(task)
        elif state == 'FAILED':
            print(f"Task {task.id} failed: {status['error_message']}")
            tasks.remove(task)
        elif state == 'CANCELED':
            print(f"Task {task.id} was canceled")
            tasks.remove(task)

    if len(tasks) > 0:
        print(f"Waiting for {len(tasks)} tasks to complete...")
        time.sleep(60)  # Check every minute

# After all exports complete, download from Drive
print("\nAll exports complete! Preparing to download from Google Drive...")

# Mount Google Drive
drive.mount('/content/drive')

# Copy files from Drive to Colab
!mkdir -p /content/downloads
!cp -r "/content/drive/MyDrive/GEE_Exports/" "/content/downloads/"

# Zip all files for single download
!zip -r /content/ndvi_results.zip /content/downloads/

# Download the zip file
files.download('/content/ndvi_results.zip')

print("\nAll files downloaded in a single zip archive!")

Please upload your shapefile (as .zip containing .shp/.shx/.dbf/.prj)


Saving StudyAreanew.zip to StudyAreanew (6).zip
Shapefile loaded: /content/shapefile/StudyArea.shp
ROI converted to Earth Engine geometry

Processing year: 2025
Month: 01 ✅ Export started (Mean NDVI: 0.089)
Month: 02 ✅ Export started (Mean NDVI: 0.084)
Month: 03 ✅ Export started (Mean NDVI: 0.083)
Month: 04 ❌ Error: Image.select: Band pattern 'NDVI' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
Month: 05 ⏭️ Skipped (future date)
Month: 06 ⏭️ Skipped (future date)
Month: 07 ⏭️ Skipped (future date)
Month: 08 ⏭️ Skipped (future date)
Month: 09 ⏭️ Skipped (future date)
Month: 10 ⏭️ Skipped (future date)
Month: 11 ⏭️ Skipped (future date)
Month: 12 ⏭️ Skipped (future date)

Monitoring all export tasks...
Waiting for 3 tasks to complete...
Waiting for 3 tasks to complete...
Waiting for 3 tasks to complete...
Waiting for 3 tasks to complete...
Waiting for 3 tasks to complete...
Waiting for 3 tasks to complete...
Task WA5KN5CXJ

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


All files downloaded in a single zip archive!
